# GPipe: Easy Scaling with Micro-Batch Pipeline Parallelism ([ссылка](https://arxiv.org/pdf/1811.06965))
**О чем:** При конвеерном параллелизме MiniBatch разделяются на Microbatch, которые обрабатываются в конвеере. Может ли повысить это скорость.

**Результат:** Значительно повышение производительности. В некоторых случаях почти до линейного уровня

## Описание Алгоритма:

После того, как пользователь определяет последовательность слоев в своей сети в терминах параметров модели $w_i$, функции прямого вычисления $f_i$ и функции оценки стоимости $c_i$, GPipe разбивает сеть на K ячеек и размещает k-ю ячейку на k-м ускорителе. Коммуникационные примитивы автоматически вставляются на границы разделов, чтобы обеспечить передачу данных между соседними разделами. Во время прямого прохода GPipe сначала делит каждую мини-партию размера N на M равных микропартий, которые проходят через K девайсов. Во время обратного прохода градиенты для каждой микропартии вычисляются на основе тех же параметров модели, которые использовались для прямого прохода. В конце каждого мини-пакета накапливаются градиенты из всех M микропакетов, которые применяются для обновления параметров модели на всех ускорителях. Эта последовательность операций проиллюстрирована на рисунке `с`. Если в сети используется пакетная нормализация, то достаточная статистика входов при обучении вычисляется для каждого микропакета и для реплик, если это необходимо. Мы также отслеживаем скользящее среднее достаточной статистики по всему мини-пакету, которое будет использоваться при оценке.

![Algo](../Pictures/GPipe/GpipeAlgo.png)



### PipeDream: Fast and Efficient Pipeline Parallel DNN Training ([ссылка](https://arxiv.org/pdf/1806.03377))

**О чем:** Обзор фреймворка PipeDream для конвеерно-параллельной реализации обучения

**Результат:** Удалось значительно ускорить обучение модели, используя data + pipeline parallelism по сравнению просто с  data parallelism

## Pipeline Parallelism

**Замечание:** Framework умеет комбинировать data parallelism и pipeline parallelism (пример ниже)

![compine](../Pictures/PipeDream/Combine.png)

### Partitioning Layers Across Machines

**1) Profiling the DNN Model**

механизм профилирования использует тот факт, что обучение DNN показывает небольшую вариативность во времени вычислений и обмена данными между минипакетами. PipeDream записывает три величины для каждого слоя $l$: 
1) $T_l$, общее время вычисления на прямом и обратном проходе для слоя
2) $a_l$, размер выходных активаций слоя (В том числе размер входных градиентов для обратного прохода)
3) $w_l$, размер параметров слоя $l$.

Чтобы определить $T_l$ для всех слоев, PipeDream профилирует короткий прогон модели DNN с использованием 1000 минипакетов на одной из машин\*. Используя этот профиль PipeDream вычисляет $T_l$ как сумму времени прямых и обратных вычислений для слоя $l$. Вся коммуникация происходит в три этапа: 
1) перемещение данных от графического процессора к центральному процессору отправителя
2) отправка данных от отправителя к получателю по сети
3) перемещение данных от центрального процессора к графическому процессору получателя. 

PipeDream оценивает время, затраченное на коммуникацию, как объем данных, который необходимо передать, разделенный на пропускную способность сети на канале связи. $C_l$ -  время, затраченное на передачу активаций от уровня $l$ к $l + 1$ в конвейере, оценивается с помощью $a_l$. Объем данных, передаваемых на одного воркера с параллелизмом данных на $m$ машинах, составляет $4 × (m − 1) × |w_l |/m$; это используется для оценки $W_l^m$ - времени синхронизации веса для слоя при использовании сервера распределенных параметров.

**2) Partitioning Algorithm**

Наш алгоритм секционирования берет выходные данные шага профилирования и вычисляет: 
1) разделение слоев на этапы
2) коэффициент репликации для каждого этапа 
3) оптимальное количество мини-пакетов для обеспечения занятости конвейера обучения. 

Алгоритм секционирования пытается свести к минимуму общее время обучения модели. Для конвейерной системы эта проблема эквивалентна минимизации времени, затрачиваемого на самый медленный этап конвейера. Эта задача обладает свойством sub-problem property\*\*; Конвейер, который максимизирует пропускную способность при заданном количестве компьютеров, состоит из подконвейеров, которые максимизируют пропускную способность при меньшем количестве машин. Следовательно, мы можем найти оптимальное решение этой проблемы с помощью динамического программирования. Пусть $A(j, m)$ - время, затраченное на самую медленную стадию в оптимальном конвейере между уровнями $1$ и $j$ с использованием $m$ машин. Цель нашего алгоритма — найти $A(N, M)$ и соответствующее разбиение. Пусть $T(i → j, m)$ обозначает время, затраченное на одну стадию, охватывающую слои от $i$ до $j$, реплицированную на $m$ машинах.

$$
T(i → j, m) = \frac{1}{m}\max(\sum_{l=i}^jT_l, \sum_{l=i}^jW_l^m)
$$

где левый член внутри $\max$ — это общее время вычислений для всех слоев в рабочей области, а правый член — общее время связи для всех слоев в рабочей области.
Оптимальный конвейер, состоящий из слоев от $1$ до $j$ с использованием $m$ машин, может быть либо одним этапом, повторяемым $m$ раз (просто data parallelism), либо состоять из нескольких этапов (pipeline + data parallelism).

* Случай 1: data parallelism 

$$A(j, m) = T(1 → j, m)$$
* Случай 2: data + pipeline parallelism

Оптимальный конвейер содержит более одного этапа. В этом случае его можно разбить на оптимальный подконвейер, состоящий из слоев от $1$ до $i$ с $m−m'$ машинами, за которыми следует один этап со слоями от $i + 1$ до $j$, реплицированных на $m'$ машин. Затем, используя свойство оптимальной подзадачи:

$$A(j,m) = \min_{1 \leq i < j}\min_{1 \leq m' < m}\max(A(i, m-m'), 2C_i, T(i+1 \rightarrow j, m'))$$

где первый член внутри $\max$ — это время, затраченное на самую медленную стадию оптимального подконвейера между уровнями $1$ и $i$ с $m − m'$ машинами, второй член — время, затраченное на передачу активаций и градиентов между слоями $i$ и $i + 1$, а третий член — время, затраченное на одну ступень, содержащую оставшиеся слои в конфигурации с параллельной передачей данных машин $m'$.

**Инициализация:** 

$A(1, m) := T(1 → 1, m)$, где $T(.)$ соответствует определению выше, а $m$ изменяется от $1$ до $M$ (общее число машин). $A(i, 1) := T(1 → i, 1)$, где $i$ изменяется от $1$ до $N$ (общее количество слоев в модели).

**Анализ сложности:**

Общее число подзадач равно $O(NM)$. Временная сложность для каждой подзадачи также равна $O(NM)$, что приводит к общей временной сложности $O(N^2M^2). На основе разбиения, сгенерированного нашим алгоритмом, оптимальное количество минипакетов, допущенных на каждом входном этапе для поддержания конвейера в стабильном состоянии, определяется как `[(# machines) / (# machines in the input stage)]`. Мы называем эту величину `NUM_OPT_ACTIVE_MINIBATCHES` (NOAM)

**Замечание:**

\* Все графические процессоры, используемые в отдельных экспериментах, идентичны. В результате достаточно профилировать производительность на одном графическом процессоре.
\*\*Свойство sub-problem property (свойство подпроблемы) - это ключевая характеристика задач, которые могут быть решены с помощью динамического программирования. Оно означает, что задача может быть разбита на подзадачи, решения которых используются повторно во время вычисления полного решения.

### Work Schreduling

В отличие от традиционных однонаправленных конвейеров, конвейерное обучение DNN включает в себя двунаправленный конвейер. Прямой проход для мини-пакета начинается на входном слое, а обратный проход заканчивается на нем. Следовательно, каждая активная мини-партия в конвейере может находиться на разных уровнях либо в прямом, либо в обратном проходе. В результате, каждая машина в системе должна сделать выбор между двумя вариантами:
1) выполнить прямой проход для минипакета, тем самым перемещая минипакет к нижестоящим машинам
2) выполнить обратный проход для другого минипакета, тем самым обеспечив поступательный прогресс в обучении.

Простой механизм планирования, который всегда отдает приоритет прямому проходу, препятствует общему прогрессу, так как обновления весов могут быть применены только после завершения обратных проходов. Точно так же, постоянное приоритизирование работы в обратном направлении может периодически приводить к тому, что машины будут простаивать без доступа к работе. Мы предлагаем механизм планирования, который позволяет избежать этих проблем.

На этапе ввода входной каскад пропускает мини-пакеты NOAM, чтобы поддерживать конвейер в стабильном состоянии. После перехода в устойчивое состояние каждый этап чередуется между выполнением прямого и обратного прохода для мини-пакета. Мы называем этот механизм «один вперед-один назад» (1F1B). В сбалансированном конвейере 1F1B гарантирует, что ни один графический процессор не простаивает в стабильном состоянии, и что мы продвигаемся вперед в обучении на каждом мини-пакете.

На рисунке ниже показана соответствующая временная шкала вычислений для конвейера с 4 этапами, каждый из которых выполняется на одной машине. NOAM для этой конфигурации равен 4. На этапе запуска входной каскад пропускает ровно четыре мини-пакета, которые распространяются до выходного каскада. Как только этап вывода завершает прямой проход для первого минипакета, он выполняет обратный проход для того же минипакета, а затем начинает чередовать выполнение прямых и обратных проходов для последующих минипакетов. 
По мере того как обратный проход начинает распространяться на более ранние этапы конвейера, каждый этап начинает чередоваться между прямым и обратным проходом для разных мини-пакетов. Как показано на рисунке, в установившемся состоянии каждая машина занята выполнением либо прямого прохода, либо обратного прохода для мини-партии. Для того, чтобы 1F1B был эффективным, не обязательно, чтобы проход вперед занимал столько же времени, сколько проход назад. На самом деле, мы видим, что на практике обратный проход всегда дольше, чем прямой проход, и 1F1B остается эффективным механизмом планирования.

Когда этапы выполняются в конфигурации с параллельными данными, реплицируемой между несколькими графическими процессорами, мы используем детерминированную балансировку нагрузки по циклическому перебору (`minibatchID mod stageReplicaID`) для распределения работы с предыдущих этапов по репликам. Такая детерминированная балансировка нагрузки гарантирует, что обратный проход для мини-партии выполняется на машине, ответственной за прямой проход минипартии. Как стратегия планирования 1F1B для этапов в конвейере, так и стратегия циклической балансировки нагрузки между реплицированными этапами являются статическими стратегиями. Таким образом, они могут выполняться каждой машиной (здесь имеется ввиду этам в конвеере) независимо друг от друга, не требуя дорогостоящей распределенной координации.

![Schreduler](../Pictures/PipeDream/Schreduler.png)

## Effective learning

В наивно конвейерной системе прямой проход для каждой мини-партии выполняется с использованием одной версии параметров, а обратный проход — с использованием другой версии параметров. На рисунке выше показано это с помощью секционирования без параллелизма данных. Если мы наблюдаем стадию 1 (машина 1), прямой проход для минипакета 5 выполняется после применения обновлений из минипакета 1, тогда как обратный проход для минипакета 5 выполняется после применения обновлений из минипакетов 2, 3 и 4. В результате, при обратном проходе для мини-пакета 5 на этапе 1 градиент вычисляется с использованием другого набора весов, чем тот, который используется в соответствующем прямом проходе; Это расхождение в весовых версиях может помешать сходимости модели. Кроме того, разные этапы модели DNN страдают от разной степени устаревания. Например, на третьем этапе каждый минипакет имеет только одно чередующееся обновление между прямым и обратным проходом, в то время как на выходном этапе чередование обновлений отсутствует. Эта асимметрия между слоями может еще больше повлиять на сходимость модели. Наши экспериментальные результаты показывают, что наивная конвейерная обработка не обеспечивает такой же точности, как параллельное обучение данных. Чтобы решить эту проблему, PipeDream использует два метода.

**1) Weight Stashing**

Функция Weight Stashing поддерживает несколько версий гирь, по одной для каждой активной минипартии. При выполнении прямого прохода каждая ступень обрабатывает мини-партию с использованием последней версии доступных весов. После завершения прямого прохода PipeDream сохраняет веса, используемые как часть промежуточного состояния для этой мини-партии. При выполнении обратного прохода мини-партии для вычисления градиента веса используется та же версия весов. Weight stashing гарантирует, что в рамках этапа для прямого и обратного прохода данной мини-партии используется одна и та же версия параметров модели. Например, на рисунке  выше minibatch 5 использует обновления параметров из пакета 1 на машине 1 и из 2 на машине 2. Weight stashing ничего не говорит о согласованности версий параметров, используемых для данной мини-партии на разных этапах

**2) Wertical Sync**

Вертикальная синхронизация устраняет потенциальное несоответствие между этапами. Например, на рисунке выше, используя вертикальную синхронизацию, minibatch 5 использует параметры, обновленные minibatch 1 на всех машинах как для прямых, так и для обратных проходов. Каждая мини-партия $(m_i)$, поступающая в конвейер, связана с последней версией веса $(W^{(i−x)})$, видимой на этапе ввода. Эта информация распространяется вместе с активациями и градиентами по мере того, как минибатч $m_i$ проходит по конвейеру в прямом направлении. На всех этапах прямого прохода для $m_i$ использует сохраненные веса $W^{(i−x)}$, в отличие от последнего обновления веса. После выполнения обратного прохода для $m_i$ (с использованием сохраненных весов $w^{(i−x)}$ каждый этап независимо применяет обновления веса для создания последних весов $w^(i)$), а затем может быть удален $w^{(i-x)}$. Эта координация между этапами является асинхронной.

**Stateless**

Теперь мы можем формализовать степень устаревания весовых обновлений для каждого из этих методов. В этом обсуждении мы предполагаем прямой конвейер с моделью, разбитой на $n$ этапов; Веса на каждом этапе представлены как $w_1, w_2$ и так далее. Кроме того, мы обозначим $w^{(t)}_1$ как веса $w_1$ после $t$ мини-партий. Теперь, после каждой минипартии, мы вычисляем градиент $\nabla f(w_1, w_2, ..., w_n)$, усредненный по всем образцам в минипартии. Ванильный мини-пакет $SGD$ ( $f$ — функция потерь, которую мы пытаемся оптимизировать, а $\nu$ — скорость обучения) имеет следующее обновление градиента:

$$W^{(t+1)} = W^{(t)} − \nu · \nabla f (w^{(t)}_1 , w^{(t)}_2 ,..., w^{(t)}_n)  $$

С учетом weight stashing градиенты на стадии 1 вычисляются с весами с задержкой на $n$ шагов, градиенты для стадии 2 вычисляются с весами с задержкой на $n − 1$ шагов и так далее. Математически это означает, что наше обновление веса выглядит следующим образом: 

$$W^{(t+1)} = W^{(t)} − \nu · \nabla f (w^{(t−n+1)}_1, w^{(t−n+2)}_2 ,..., w^{(t)}_n )$$

Без припрятания веса обновление веса не является валидным градиентом функции потерь $f$ для любого вектора веса $w_1, w_2,.., w_n$. Добавление вертикальной синхронизации изменяет обновление веса на:

$$W^{(t+1)} = W^{(t)} − \nu · \nabla f (w^{(t−n+1)}_1 , w^{(t−n+1)}_2 ,..., w^{(t−n+1)}_n )$$

Это семантически то же самое, что и параллелизм данных при синхронизации BSP на $n$ машинах (с одинаковым исходным размером мини-партии на каждой машине). 

**Замечание:** По сути мы в обоих случаях сохраняем веса до тех пор, пока у нас не выйдет полностью (с обратным проходом) minibatch


# Seq1F1B: Efficient Sequence-Level Pipeline Parallelism for Large Language Model Training ([ссылка](https://arxiv.org/html/2406.03488v1))

Чтобы получить более эффективное расписание конвейера 1F1B на уровне последовательности, предлагается Seq1F1B, который представляет собой расписание 1F1B, созданное вручную для ввода на уровне последовательности. В частности, Seq1F1B разбивает модель на последовательные наборы слоев и назначает каждому исполнителю соответствующий набор (так называемые этапы конвейера). Затем Seq1F1B инициализирует часть расписания. Как и в 1F1B, расписание разделено на три фазы: прогрев, стабилизация и заминка.

![seq1F1B](../Pictures/seq1F1BAndZeroBubble/seq1F1B.png)

В 1F1B-I каждому рабочему назначается несколько стадий. 

![seq1F1B-I](../Pictures/seq1F1BAndZeroBubble/seq1F1B-I.png)

Традиционно нейронные сети гранулируются в виде стековых слоев. С каждым слоем связаны две функции: прямая и обратная. При прямом проходе вход x преобразуется в выходной y с параметризованным отображением f(x,W). Обратный пасс, критически важный для обучения, включает в себя два вычисления: $\nabla_xf(x,W)^T$ и $\nabla_Wf(x,W)^T$. Собственно, в этом и вся идея этого планировщика.

![ZeroBubble](../Pictures/seq1F1BAndZeroBubble/ZeroBubble.png)

объединение с seq1F1B с ZB-H1

![ResultSchreudler](../Pictures/seq1F1BAndZeroBubble/ZeroBubbleAndseq1F1B.png)


Подробно рассмотрим стратегию секционирования последовательности и балансировки рабочей нагрузки. Для эффективного планирования конвейера крайне важно, чтобы время обработки для каждой подпоследовательности было примерно одинаковым, чтобы избежать образования пузырей конвейера. Исходя из этой предпосылки, мы разрабатываем вычислительную стратегию разбиения, оценивая FLOPs последовательностей и создавая теоретическое решение, направленное на максимальное сближение FLOPs всех подпоследовательностей.


Для входной последовательности $S=(x_1, x_2, ..., x_n)$ мы делим его на $k$ сегментов S = \[S_1, ..., S_k\] Каждый сегмент имеет длину $n_i$, где $\sum\limits_{i=1}^nn_i=n$. Ожидается, что вычислительный объем будет примерно одинаков:
$$
FLOPs(S_1)=...=FLOPs(S_k)=\frac{FLOPs(S)}{k}
$$

Для оценки FLOPs используется метод Хофмана:
$$
FLOPs(S_i)=2n_iP+2Ln_i(\sum\limits_{j=0}^in_j)d \ \forall i = 1..k \\
FLOPs(S)=2nP+2Ln^2d
$$
где $L$ - количество слоев модели, $d$ - размерность модели(размерность вектора активаций), $P$ - общее число параметров

### Результаты

![Results](../Pictures/seq1F1BAndZeroBubble/Results.png)